In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Importing data

In [2]:
train_dataset  =pd.read_csv('/kaggle/input/titanic/train.csv')
test_dataset = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
train_dataset =train_dataset.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
test_passengerIds = test_dataset['PassengerId'].values
test_dataset=test_dataset.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)


In [4]:
train_dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [5]:
train_dataset = pd.get_dummies(train_dataset, columns = ['Sex','Embarked'])
test_dataset = pd.get_dummies(test_dataset, columns = ['Sex','Embarked'])

In [6]:
train_dataset.head()


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [7]:
test_dataset.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,1,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,0,1
2,2,62.0,0,0,9.6875,0,1,0,1,0
3,3,27.0,0,0,8.6625,0,1,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,0,1


# Checking null values

In [8]:
train_dataset.isnull().sum()

Survived        0
Pclass          0
Age           177
SibSp           0
Parch           0
Fare            0
Sex_female      0
Sex_male        0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [9]:
test_dataset.isnull().sum()

Pclass         0
Age           86
SibSp          0
Parch          0
Fare           1
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

# Filling null values

In [10]:
train_dataset['Age'].fillna(train_dataset['Age'].mean(),inplace = True)
test_dataset['Age'].fillna(test_dataset['Age'].mean(),inplace = True)
test_dataset['Fare'].fillna(test_dataset['Fare'].mean(),inplace = True)

In [11]:
train_dataset.isnull().sum()

Survived      0
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [12]:
test_dataset.isnull().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

# Splitting traning set

In [13]:
X_train = train_dataset.iloc[:,1:].values
y_train=train_dataset['Survived'].values

In [14]:
print(X_train)

[[ 3.         22.          1.         ...  0.          0.
   1.        ]
 [ 1.         38.          1.         ...  1.          0.
   0.        ]
 [ 3.         26.          0.         ...  0.          0.
   1.        ]
 ...
 [ 3.         29.69911765  1.         ...  0.          0.
   1.        ]
 [ 1.         26.          0.         ...  1.          0.
   0.        ]
 [ 3.         32.          0.         ...  0.          1.
   0.        ]]


# Splitting test set

In [15]:
 X_test = test_dataset.values

In [16]:
print(X_test)

[[ 3.         34.5         0.         ...  0.          1.
   0.        ]
 [ 3.         47.          1.         ...  0.          0.
   1.        ]
 [ 2.         62.          0.         ...  0.          1.
   0.        ]
 ...
 [ 3.         38.5         0.         ...  0.          0.
   1.        ]
 [ 3.         30.27259036  0.         ...  0.          0.
   1.        ]
 [ 3.         30.27259036  1.         ...  1.          0.
   0.        ]]


# Feature scaling

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Training the Kernel SVM model on the Training set

In [18]:
from sklearn.svm import SVC

classifier= SVC(kernel='rbf')
classifier.fit(X_train, y_train)

SVC()

# Predicting test values

In [19]:
y_pred = classifier.predict(X_test)

# Creating output file

In [20]:
output = pd.DataFrame({'PassengerId':test_passengerIds, 'Survived': y_pred})
output.to_csv('submission.csv', index=False)